<h1>PyCrypt</h1>

SOME OPENING WORDS 123.

<h3>Table of Contents</h3>

1. [Introduction](#introduction)
2. [Character Mapping](#maps)
3. [Adding, Accessing, and Using Ciphers](#ciphers)
4. [Security Features](#security)

<h1>Introduction</h1><a name="intro></a>


<h2><b>database.Init()</b></h2>

This function is automatically called when importing <code>database</code> and is
responsible for establishing a connection to PyCrypt. If the
necessary database or user does not exist, they will be created

<h2><b>database.con</b></h2>

The <code>pg8000.native.Connection</code> object created by calling
<code>database.Init()</code>. This is used to interact with the PyCrypt
Database. See [pg8000 documentaion](https://github.com/tlocke/pg8000) for more information


In [2]:
import database
import core

<h1> Character Mapping </h1> <a name="maps"></a>

<h2><b>database.SaveMap</b>(<i>name, transform, inverse, keywords</i>)</h2>

<DL>
<DT><i>name</i>
<DD>The name of the cipher to be added to the database. Will raise an exception the passed name is already taken.
<DT><i>transfom</i>
<DD>A <code>dict[str, int]</code> defining what value each unicode character maps to.
<DT><i>inverse</i>
<DD> A <code>dict[int, int]</code> used to reverse the mapping process. if inverse is <code>None</code>, then one will be generated.
<DT><i>keywords</i>
<DD>A <code>list[str]</code> containing keywords that can be sorted by once cipher is added to database.
</DL>

In [5]:
if len(database.con.run("""SELECT 1 FROM maps WHERE "name"='alphaLower'""")) == 0:
    transform = {
        "A":0,  "a":0,  "B":1,  "b":1,  "C":2,  "c":2,  "D":3,  "d":3,  "E":4,  "e":4,
        "F":5,  "f":5,  "G":6,  "g":6,  "H":7,  "h":7,  "I":8,  "i":8,  "J":9,  "j":9,
        "K":10, "k":10, "L":11, "l":11, "M":12, "m":12, "N":13, "n":13, "O":14, "o":14,
        "P":15, "p":15, "Q":16, "q":16, "R":17, "r":17, "S":18, "s":18, "T":19, "t":19,
        "U":20, "u":20, "V":21, "v":21, "W":22, "w":22, "X":23, "x":23, "Y":24, "y":24,
        "Z":25, "z":25}

    t = core.DecompressTransform(transform)[0]
    i = core.GenInverseTransform(t)

    inverse = core.CompressInverse(i)

    keywords = ["alpha", "lower", "ascii", "alphabet"]

    database.SaveMap("alphaLower", transform, inverse, keywords)

<h2><b>database.LoadMap</b>(<i>identifier</i>)</h2>

<DL>
<DT><i>identifier</i>
<DD>This can be either a sting or an int. If identifier is an integer, <code>database.LoadMap( )</code> will return the map with that ID. Otherwise, it will return all maps whose name matches the passed identifier.
</DL>

In [7]:
query  = database.LoadMap("alphaLower")
print(f"Hash of alphaLower map: '{query[1]}'")

Hash of alphaLower map: '[1606611960, -1776562239, -1598481368, -350300958, 324321788, 1933357532, -348273046, 421523614, 257989345, -1447750970, 49653793, -973305549, -1614793560, 137473864, 1730089374, -749819799]'


<h1> Adding, Accessing and Using Ciphers </h1> <a name = "ciphers"></a>

<h2><b>database.SaveCipher</b>(<i>name, formula, inverse, keywords, options</i>)</h2>

Security checks are automatically preformed whenever this function is called.
Notably, exceptions will be raised if any security check fails. Additional
information is given in
<b>SECURITY_NOTEBOOK</b>

<DL>
<DT><i>name</i>
<DD>The name of the cipher to be added to the database. Will raise an exception the passed name is already taken.
<DT><i>formula</i>
<DD>A string consisting of valid python code that transforms a mapped plaintext array into cypher text. The locals <code>maskedIndices</code>, <code>mappedIndices</code> and <code>mapRange</code> are supplied to the formula during execution. Additionally, <code>numpy</code> is automatically imported as <code>np</code> at run time. The names <code>__func_name__</code> , <code>__var_name__</code> , and <code>__class_instance__</code> are reserved and will raise exceptions if used within the formula's text.
<DT><i>inverse</i>
<DD>Python code that transforms cypher text into plaintext. The rules for the <code>formula</code> parameter apply.
<DT><i>keywords</i>
<DD>A list containing keywords that can be sorted by once cipher is added to database.
<DT><i>options</i>
<DD>A dictionary in which each key/value pair represents the name and default value of extra locals passed to cipher/inverse. 
</DL>

In [2]:
# NOTE: This only runs if a vigenere cipher is not aready in the system
if len(database.con.run("""SELECT 1 FROM ciphers WHERE "name"='vigenere'""")) == 0:

    keywords = ["vigenere","caesar","polyalphabetic"]
    options = {"cycleKeywordOutsideMap":False, "deleteTextOutsideMap":True}

    formulaStr = """
    assert len(mapRange) == 1 + max(mapRange)

    if options["cycleKeywordOutsideMap"]:
        offset = np.resize(keys[0], len(text))[mappedIndices]

    else:
        offset = np.resize(keys[0], len(mappedIndices))

    out = (text[mappedIndices] + offset) % len(mapRange)

    if not options["deleteTextOutsideMap"]:
        out = np.put(text, mappedIndices, out)
    """

    inverseStr = """
    assert len(mapRange) == 1 + max(mapRange)

    if options["cycleKeywordOutsideMap"]:
        offset = np.resize(keys[0], len(text))[mappedIndices]
    else:
        offset = np.resize(keys[0], len(mappedIndices))

    if options["deleteTextOutsideMap"]:
        out = (text - offset) % len(mapRange)
    else:
        out = (text[mappedIndices] - offset) % len(mapRange)
        out = np.put(text, mappedIndices, out)
    """

    database.SaveCipher("vigenere", formulaStr, inverseStr, keywords, options)

<h2><b>database.LoadCipher</b>(<i>identifier</i>)</h2>

<DL>
<DT><i>identifier</i>
<DD>This can be either a sting or an int. If identifier is an integer, <code>database.LoadCipher( )</code> will return the cipher with that ID. Otherwise, it will return all ciphers whose name matches the passed identifier.
</DL>

In [3]:
query  = database.LoadCipher("vigenere")
print(f"Hash of vigenere cipher: '{query[1]}'")


Hash of first cipher: '[-679745061, 1038564999, -428354572, -412549081, -2071110137, -1827022449, 2001695571, 638559363, 595319930, 1491456457, 571760906, 1964182341, 744083102, -1494954262, 1632359335, -1648850155]'


<h1> Security Features</h1> <a name = "security"></a>